## Nasa Numeric

In [9]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 

from sklearn import model_selection as ms
from sklearn.datasets import make_regression

In [13]:
#Carregando dados

data, meta = arff.loadarff('datasets/nasa_numeric.arff')
df = pd.DataFrame(data)
df.head()

,recordnumber,projectname,cat2,forg,center,year,mode,rely,data,cplx,...,acap,aexp,pcap,vexp,lexp,modp,tool,sced,equivphyskloc,act_effort
0,1.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',25.9,117.6
1,2.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',24.6,117.6
2,3.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',7.7,31.2
3,4.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',8.2,36.0
4,5.0,b'de',b'avionicsmonitoring',b'g',b'2',1979.0,b'semidetached',b'h',b'l',b'h',...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',9.7,25.2


## Preparando Dados

In [16]:
def one_hot_enconder(X_transform, transform_data): #X_transform = data, transform_data = colunas
    
    for col in transform_data:
        cols = X_transform[col].unique() #retorna os valores da categoria
        col_position = X_transform.columns.get_loc(col) #retorna posição da coluna
        col_values = X_transform.pop(col) #retorna e apaga coluna
        for c in cols: 
            X_transform.insert(col_position, str(col) + "_" + str(c), col_values.eq(c))
            col_position += 1

    return X_transform.replace({True: 1, False: 0})          

,recordnumber,projectname,cat2_b'avionicsmonitoring',cat2_b'missionplanning',cat2_b'simulation',cat2_b'monitor_control',cat2_b'realdataprocessing',cat2_b'communications',cat2_b'batchdataprocessing',cat2_b'datacapture',...,acap,aexp,pcap,vexp,lexp,modp,tool,sced,equivphyskloc,act_effort
0,1.0,b'de',1,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',25.90,117.6
1,2.0,b'de',1,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',24.60,117.6
2,3.0,b'de',1,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',7.70,31.2
3,4.0,b'de',1,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',8.20,36.0
4,5.0,b'de',1,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'n',b'h',b'h',b'n',b'l',9.70,25.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,97.0,b'gal',0,0,0,0,0,0,0,0,...,b'h',b'l',b'n',b'vl',b'l',b'l',b'h',b'h',7.25,648.0
89,98.0,b'Y',0,0,0,0,0,0,0,0,...,b'h',b'h',b'h',b'h',b'h',b'h',b'n',b'h',233.00,8211.0
90,99.0,b'X',0,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'l',b'l',b'n',b'n',b'h',16.30,480.0
91,100.0,b'X',0,0,0,0,0,0,0,0,...,b'n',b'n',b'n',b'l',b'l',b'n',b'n',b'h',6.20,12.0


In [ ]:
#removendo atributos desnecessários "recordnumber", "projectname", "year".

df = df.drop(['recordnumber','projectname', 'year'], axis=1)

#removendo linhas com atributos nulos ou vazios

df = df.dropna()

#Convertendo atributos textuais em numeric com escala de atributos (ainda sem normalização) "OrdinalEncoder"

tranform_data = {b"vl": 0,b"l":1,b"n":2,b"h":3,b"vh":4,b"xh":5}
df = df.replace(tranform_data)

#Convertendo atributos categóricos em colunas. "OneHotEncoder"




#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

#df = (df-df.mean())/df.std()

#separando atributo classe Effort para entrada nos modelos

#X = df.drop(df.columns[-1], axis=1)
#y = df.take([-1], axis=1)

## Treinando e avaliando o desempenho dos modelos

In [ ]:
models = []
#Instanciando os modelos
models.append(['KNN', KNeighborsRegressor(n_neighbors=5)]) 
models.append(['MLP', MLPRegressor(activation='logistic', solver='sgd', alpha=0.02, max_iter=300, hidden_layer_sizes=200)])
models.append(['RFR', RandomForestRegressor()])
models.append(['SVR', svm.SVR()])

In [ ]:
for model in models:
    cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
    scores = ms.cross_val_score(model[1], X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    #converter para positivo.
    scores = np.absolute(scores)
    s_mean = np.mean(scores)
    s_std = np.std(scores)
    
    print('-', model[0],'------------------------------------')
    print('Scores:', scores, 'MAE: %.4f' % (s_mean), 'STD: %.4f' % (s_std)) 